# Vertex Smoothing

Smoothing out or smoothing a vertex ***w*** with regards to the pair of edges (e1, e2) incident on ***w***, removes both edges containing ***w*** and replaces (e1, e2) with a new edge that connects the other endpoints of the pair.

For example, the simple connected graph with two edges, e1 {u,w} and e2 {w,v}:
![alt text](https://upload.wikimedia.org/wikipedia/commons/6/6f/Graph_subdivision_step2.svg "Graph 1")

has a vertex (namely w) that can be smoothed away, resulting in:
![alt text](https://upload.wikimedia.org/wikipedia/commons/1/15/Graph_subdivision_step1.svg "Graph 1")


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from graphy.smoothing.vertexsmoothing import vertex_smoothing

## Setup

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
spark

## Utility Functions

In [5]:
# Utils functions ============================
# TUPLE

def source_getter_tuple(x):
    return x[0]

def target_getter_tuple(x):
    return x[2]

def link_getter_tuple(x):
    return x[1]

def obj_creator_tuple(s, l, t):
    return (s, l, t)


# DICT
def source_getter_dict(x):
    return x['SOURCE']

def target_getter_dict(x):
    return x['TARGET']

def link_getter_dict(x):
    return x['LINK']

def obj_creator_dict(s, l, t):
    return {'SOURCE': s, 'LINK': l, 'TARGET': t}

Some example of usage...

In [6]:
tup = ('from', 'transform', 'to')
diz = {'SOURCE': 'from', 'LINK': 'transform', 'TARGET': 'to'}

print("TUPLE =================================================")
print(source_getter_tuple(tup))
print(target_getter_tuple(tup))
print(link_getter_tuple(tup))
print(obj_creator_tuple('sss', 'new_link', 'ttt'))
print("DICT ==================================================")
print(source_getter_dict(diz))
print(target_getter_dict(diz))
print(link_getter_dict(diz))
print(obj_creator_dict('sss', 'new_link', 'ttt'))

TUPLE =================================================
from
to
transform
('sss', 'new_link', 'ttt')
DICT ==================================================
from
to
transform
{'SOURCE': 'sss', 'LINK': 'new_link', 'TARGET': 'ttt'}


## First trivial example

Let's try a trivial example.

we have the following links:

    T_ORIGINAL_DATA -> V_DATA -> T_DATA_WORK -> T_DATA_FINAL

and we want to end up with:
    
    T_ORIGINAL_DATA -> T_DATA_FINAL.

We have to remove the two vertices in the middle.

In [7]:
# T_ORIGINAL_TABLE -> V_TABLE -> T_TABLE_WORK -> T_TABLE_FINAL
# should become
# T_ORIGINAL_TABLE -> T_TABLE_FINAL

vertices_list = [
    ('T_ORIGINAL_DATA',),
    ('V_DATA',),
    ('T_DATA_WORK',),
    ('T_DATA_FINAL',)
]

links_list = [
    ('T_ORIGINAL_DATA', 'pt', 'V_DATA'),
    ('V_DATA', 'si', 'T_DATA_WORK'),
    ('T_DATA_WORK', 'pt', 'T_DATA_FINAL')
]

vertices_df = spark.createDataFrame(vertices_list, ['ITEM'])
links_df = spark.createDataFrame(links_list, ['SOURCE', 'LINK', 'TARGET'])

vertices_df.show()
links_df.show()

+---------------+
|           ITEM|
+---------------+
|T_ORIGINAL_DATA|
|         V_DATA|
|    T_DATA_WORK|
|   T_DATA_FINAL|
+---------------+

+---------------+----+------------+
|         SOURCE|LINK|      TARGET|
+---------------+----+------------+
|T_ORIGINAL_DATA|  pt|      V_DATA|
|         V_DATA|  si| T_DATA_WORK|
|    T_DATA_WORK|  pt|T_DATA_FINAL|
+---------------+----+------------+



### RDD as the collection data model

In [9]:
vertices_rdd = vertices_df.rdd.map(lambda x: x.asDict())
links_rdd = links_df.rdd.map(lambda x: x.asDict())

vertices_to_remove = (vertices_rdd
                      .map(lambda x: x['ITEM'])
                      .filter(lambda x: x.startswith('V') or x.rfind('WORK') != -1)
                      .collect()
                      )
vertices_cleaned_rdd = vertices_rdd.filter(lambda x: x['ITEM'] not in vertices_to_remove)

links_cleaned_rdd = vertex_smoothing(
    links_rdd.repartition(1), vertices_to_remove, source_getter_dict, target_getter_dict,
    link_getter_dict, obj_creator_dict
)
print("RDD -> Links cleaned:", links_cleaned_rdd.collect())

RDD -> Links cleaned: [{'SOURCE': 'T_ORIGINAL_DATA', 'LINK': 'pt-si-pt', 'TARGET': 'T_DATA_FINAL'}]


In [10]:
print(links_cleaned_rdd.toDebugString().decode('UTF-8'))

(6) UnionRDD[42] at union at NativeMethodAccessorImpl.java:0 []
 |  PythonRDD[40] at RDD at PythonRDD.scala:49 []
 |  UnionRDD[36] at union at NativeMethodAccessorImpl.java:0 []
 |  PythonRDD[34] at RDD at PythonRDD.scala:49 []
 |  MapPartitionsRDD[30] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  CoalescedRDD[29] at coalesce at NativeMethodAccessorImpl.java:0 []
 |  ShuffledRDD[28] at coalesce at NativeMethodAccessorImpl.java:0 []
 +-(4) MapPartitionsRDD[27] at coalesce at NativeMethodAccessorImpl.java:0 []
    |  PythonRDD[26] at RDD at PythonRDD.scala:49 []
    |  MapPartitionsRDD[23] at javaToPython at NativeMethodAccessorImpl.java:0 []
    |  MapPartitionsRDD[22] at javaToPython at NativeMethodAccessorImpl.java:0 []
    |  MapPartitionsRDD[21] at javaToPython at NativeMethodAccessorImpl.java:0 []
    |  MapPartitionsRDD[9] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:0 []
    |  MapPartitionsRDD[8] at map at SerDeUtil.scala:137 []
    |  MapPartitionsRDD[7] a

In [11]:
links_cleaned_rdd.getNumPartitions()

6

### LIST as the collection data model

In [12]:
vertices_to_remove = filter(lambda x: x.startswith('V') or x.rfind('WORK') != -1,
                            map(lambda x: x[0], vertices_list))
vertices_cleaned_list = filter(lambda x: x[0] not in vertices_to_remove, vertices_list)

links_cleaned_list = vertex_smoothing(
    links_list, vertices_to_remove, source_getter_tuple, target_getter_tuple,
    link_getter_tuple, obj_creator_tuple
)
print("LIST -> Links cleaned:", links_cleaned_list)

LIST -> Links cleaned: [('T_ORIGINAL_DATA', 'pt-si-pt', 'T_DATA_FINAL')]


## Second (not trivial) example

Let's try a less trivial example.

We have the following links:

     T_ORIGINAL_TABLE_1 -\                            /-> T_TABLE_FINAL_1 
                          -> V_TABLE -> T_TABLE_WORK -
     T_ORIGINAL_TABLE_2 -/                            \-> T_TABLE_FINAL_2

and we want to end up with:

     T_ORIGINAL_TABLE_1 -\ /-> T_TABLE_FINAL_1 
                          X
     T_ORIGINAL_TABLE_2 -/ \-> T_TABLE_FINAL_2

We have to remove the two vertices in the middle, and build more complex links.

In [13]:
# T_ORIGINAL_TABLE_1 -\                            /-> T_TABLE_FINAL_1 
#                      -> V_TABLE -> T_TABLE_WORK -
# T_ORIGINAL_TABLE_2 -/                            \-> T_TABLE_FINAL_2
#
# should become
#
# T_ORIGINAL_TABLE_1 -\ /-> T_TABLE_FINAL_1 
#                      X
# T_ORIGINAL_TABLE_2 -/ \-> T_TABLE_FINAL_2


vertices_list = [
    ("T_ORIGINAL_TABLE_1",),
    ("T_ORIGINAL_TABLE_2",),
    ("V_TABLE",),
    ("T_TABLE_WORK",),
    ("T_TABLE_FINAL_1",),
    ("T_TABLE_FINAL_2",)
]

links_list = [
    ('T_ORIGINAL_TABLE_1', 'mi', 'V_TABLE'),
    ('T_ORIGINAL_TABLE_2', 'mi', 'V_TABLE'),
    ('V_TABLE', 'pt', 'T_TABLE_WORK'),
    ('T_TABLE_WORK', 'si', 'T_TABLE_FINAL_1'),
    ('T_TABLE_WORK', 'si', 'T_TABLE_FINAL_2')
]

vertices_df = spark.createDataFrame(vertices_list, ["ITEM"])
links_df = spark.createDataFrame(links_list, ["SOURCE", "LINK", "TARGET"])

vertices_df.show()
links_df.show()

+------------------+
|              ITEM|
+------------------+
|T_ORIGINAL_TABLE_1|
|T_ORIGINAL_TABLE_2|
|           V_TABLE|
|      T_TABLE_WORK|
|   T_TABLE_FINAL_1|
|   T_TABLE_FINAL_2|
+------------------+

+------------------+----+---------------+
|            SOURCE|LINK|         TARGET|
+------------------+----+---------------+
|T_ORIGINAL_TABLE_1|  mi|        V_TABLE|
|T_ORIGINAL_TABLE_2|  mi|        V_TABLE|
|           V_TABLE|  pt|   T_TABLE_WORK|
|      T_TABLE_WORK|  si|T_TABLE_FINAL_1|
|      T_TABLE_WORK|  si|T_TABLE_FINAL_2|
+------------------+----+---------------+



### RDD as the collection data model

In [15]:
vertices_rdd = vertices_df.rdd.map(lambda x: x.asDict())
links_rdd = links_df.rdd.map(lambda x: x.asDict())

vertices_to_remove = (vertices_rdd
                      .map(lambda x: x['ITEM'])
                      .filter(lambda x: x.startswith('V') or x.rfind('WORK') != -1)
                      .collect()
                      )
vertices_cleaned_rdd = vertices_rdd.filter(lambda x: x['ITEM'] not in vertices_to_remove)

links_cleaned_rdd = vertex_smoothing(
    links_rdd.repartition(1), vertices_to_remove, source_getter_dict, target_getter_dict,
    link_getter_dict, obj_creator_dict
)
print("RDD -> Links cleaned:", links_cleaned_rdd.collect())

RDD -> Links cleaned: [{'SOURCE': 'T_ORIGINAL_TABLE_1', 'LINK': 'mi-pt-si', 'TARGET': 'T_TABLE_FINAL_1'}, {'SOURCE': 'T_ORIGINAL_TABLE_1', 'LINK': 'mi-pt-si', 'TARGET': 'T_TABLE_FINAL_2'}, {'SOURCE': 'T_ORIGINAL_TABLE_2', 'LINK': 'mi-pt-si', 'TARGET': 'T_TABLE_FINAL_1'}, {'SOURCE': 'T_ORIGINAL_TABLE_2', 'LINK': 'mi-pt-si', 'TARGET': 'T_TABLE_FINAL_2'}]


### LIST as the collection data model

In [16]:
vertices_to_remove = filter(lambda x: x.startswith('V') or x.rfind('WORK') != -1,
                            map(lambda x: x[0], vertices_list))
vertices_cleaned_list = filter(lambda x: x[0] not in vertices_to_remove, vertices_list)

links_cleaned_list = vertex_smoothing(
    links_list, vertices_to_remove, source_getter_tuple, target_getter_tuple,
    link_getter_tuple, obj_creator_tuple
)
print("LIST -> Links cleaned:", links_cleaned_list)

LIST -> Links cleaned: [('T_ORIGINAL_TABLE_1', 'mi-pt-si', 'T_TABLE_FINAL_1'), ('T_ORIGINAL_TABLE_1', 'mi-pt-si', 'T_TABLE_FINAL_2'), ('T_ORIGINAL_TABLE_2', 'mi-pt-si', 'T_TABLE_FINAL_1'), ('T_ORIGINAL_TABLE_2', 'mi-pt-si', 'T_TABLE_FINAL_2')]
